# TWAM Community Groups - Participation

In [43]:
import os 
import pandas as pd
from pathlib import Path

In [44]:
ROOT = Path('../../../')
ROOT.resolve()

RAW_DATA = ROOT / 'raw'
CASE_STUDIES_DIR = RAW_DATA / 'case-studies'
TWAM_DATA = CASE_STUDIES_DIR / 'twam' / '23-24 data for culture connect v2.xlsx'

OUT_PATH = ROOT / 'src' / 'case-study' / 'tyne-and-wear-archives-and-museums'/ '_data'

twam = pd.read_excel(TWAM_DATA)


/Users/tazminchiles/Documents/GitHub/northumbria-culture-connect/env/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [52]:
# Filter for the Community Groups we are interested in

twam_filtered = twam[
    (twam['Group Name'] == 'CNTW') &
    (twam['Type of Group'] == 'Community Group') &
    (twam['ACE Survey Monitoring Data'] == 'Health/wellbeing')
]

# Remove empty columns and clean up column names

twam_filtered = twam_filtered.dropna(axis=1, how='all')
twam_filtered.columns = (
        twam_filtered.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=True)
        .str.replace(":", "_", regex=True)
    )

twam_filtered = twam_filtered.rename(columns={
    'lea_and_district_(or_country_if_outside_england)': 'district',
    'no.19_&_over': 'participants_over_19',
    'date_(00/00/00)': 'date'
})

# Convert year_month to datetime

twam_filtered['year_month'] = pd.to_datetime(twam_filtered['year_month'], format='%Y_%m')

# Group by month and sum participants

twam_filtered = twam_filtered.groupby('year_month')['participants_over_19'].sum().reset_index()

# Convert date to decimal year for visualisation

twam_filtered['date_decimal'] = twam_filtered['year_month'].dt.year + (twam_filtered['year_month'].dt.month - 1) / 12

# Re-order columns 
twam_filtered = twam_filtered[['year_month', 'date_decimal', 'participants_over_19']]

# Output to CSV for visualisation

twam_filtered.to_csv(OUT_PATH / 'community_groups_participation.csv', index=False)





,year_month,date_decimal,participants_over_19
0,2023-04-01,2023.250000,69
1,2023-05-01,2023.333333,221
2,2023-06-01,2023.416667,32
3,2023-07-01,2023.500000,40
4,2023-08-01,2023.583333,50
5,2023-09-01,2023.666667,51
6,2023-10-01,2023.750000,32
7,2023-11-01,2023.833333,21
8,2023-12-01,2023.916667,8
